In [33]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
from tensorflow import keras
import tensorflow as tf

In [2]:
# mainPath = '../input/plant-pathology-2020-fgvc7'
main_path = './dataset'
images_path = os.path.join(main_path, 'images')
test_path = os.path.join(main_path, 'test.csv')
train_path = os.path.join(main_path, 'train.csv')

In [3]:
test_data = pd.read_csv(test_path)
train_data = pd.read_csv(train_path)

In [4]:
train_data.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [10]:
def load_image(image_id):
    file_path = image_id + ".jpg"
    image = cv2.imread(os.path.join(images_path, file_path))
    k = 0.25
    height, wide, _ = image.shape
    dim = (int(wide*k), int(height*k))
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [30]:
input_shape = load_image(test_data['image_id'][0]).shape
print(input_shape)

(341, 512, 3)


In [53]:
X_train = train_data["image_id"].apply(lambda id: os.path.join(images_path, id + '.jpg'))

In [54]:
X_train.head()

0    ./dataset\images\Train_0.jpg
1    ./dataset\images\Train_1.jpg
2    ./dataset\images\Train_2.jpg
3    ./dataset\images\Train_3.jpg
4    ./dataset\images\Train_4.jpg
Name: image_id, dtype: object

In [43]:
y_train = train_data.loc[:, 'healthy':'scab']

In [44]:
y_train.head()

,healthy,multiple_diseases,rust,scab
0,0,0,0,1
1,0,1,0,0
2,1,0,0,0
3,0,0,1,0
4,1,0,0,0


In [67]:
batch_size = 60
epochs = 20

In [78]:
def load_dataset(filename, res):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float16) / 255.0
    image = tf.image.resize(image, input_shape[:2])
    return image, res

In [85]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_train, y_train))
    .map(load_dataset)
    .repeat()
    .shuffle(256)
    .batch(batch_size)
)

In [90]:
model = keras.Sequential([
    keras.layers.Conv2D(64, (5, 5), input_shape=input_shape, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.GlobalMaxPool2D(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(y_train.shape[1], activation='softmax')
])

In [91]:
model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [92]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 337, 508, 64)      4864      
_________________________________________________________________
batch_normalization_34 (Batc (None, 337, 508, 64)      256       
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 168, 254, 64)      0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 166, 252, 64)      36928     
_________________________________________________________________
batch_normalization_35 (Batc (None, 166, 252, 64)      256       
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 83, 126, 64)       0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 81, 124, 64)      

In [93]:
model.fit(train_dataset, epochs=20, steps_per_epoch=X_train.shape[0]//batch_size)

Epoch 1/20


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_8/conv2d_42/Conv2D (defined at <ipython-input-93-db542773c6c7>:1) ]] [Op:__inference_train_function_27875]

Function call stack:
train_function


In [ ]:
model.save(path.join('.', 'models', 'model.hdf5'))